In [1]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer,text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from tensorflow.keras.callbacks import Callback ,EarlyStopping,ModelCheckpoint
from sklearn.metrics import precision_score, recall_score,accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

from sklearn.metrics import confusion_matrix,f1_score,classification_report

In [2]:
#from google.colab import drive
#drive.mount('/content/drive')

In [3]:
df = pd.read_csv(r"C:\Users\admin\Desktop\Major project\dataset\cleanDataOne.csv")


In [4]:
df.head()

,text,y
0,cocksucker before you piss around on my work,1
1,bye do not look come or think of comming back ...,1
2,fuck your filthy mother in the ass dry,1
3,get fucked up get fuckeeed up got a drink tha...,1
4,stupid peace of shit stop deleting my stuff as...,1


In [5]:
#!pip install transformers

In [6]:
num_classes=len(df.y.unique())
num_classes

2

In [7]:
# num_classes=len(df.label.unique())

In [8]:
from transformers import *
from transformers import BertTokenizer, TFBertModel, BertConfig
#initializing bert tokenizer and model
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bert_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=1)

C:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\generation_tf_utils.py:24: FutureWarning: Importing `TFGenerationMixin` from `src/transformers/generation_tf_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import TFGenerationMixin` instead.
  warnings.warn(
loading file vocab.txt from cache at C:\Users\admin/.cache\huggingface\hub\models--bert-base-uncased\snapshots\a265f773a47193eed794233aa2a0f0bb6d3eaa63\vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at C:\Users\admin/.cache\huggingface\hub\models--bert-base-uncased\snapshots\a265f773a47193eed794233aa2a0f0bb6d3eaa63\tokenizer_config.json
loading configuration file config.json from cache at C:\Users\admin/.cache\huggingface\hub\models--bert-base-uncased\snapshots\a265f773a47193eed794233aa2a0f0bb6d3eaa63\config.json
Model config BertConfig {


In [9]:
#position embedding and 
input_ids=[]
attention_masks=[]
labels=[]
for i  in range(len(df['text'])):
    if df['text'][i] is not None:
        try:
          bert_inp=bert_tokenizer.encode_plus(df['text'][i],add_special_tokens = True,max_length =100,pad_to_max_length = True,return_attention_mask = True)
          input_ids.append(bert_inp['input_ids'])
          attention_masks.append(bert_inp['attention_mask'])
          labels.append(df['y'][i])

        except Exception as e:
          pass

input_ids=np.asarray(input_ids)
attention_masks=np.array(attention_masks)
labels=np.array(labels)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
C:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [10]:
len(input_ids),len(attention_masks),len(labels)

(18054, 18054, 18054)

In [11]:
train_inp,val_inp,train_label,val_label,train_mask,val_mask=train_test_split(input_ids,labels,attention_masks,test_size=0.2)


In [12]:

print('\nBert Model',bert_model.summary())

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  769       
                                                                 
Total params: 109,483,009
Trainable params: 109,483,009
Non-trainable params: 0
_________________________________________________________________

Bert Model None


In [13]:
precision = tf.keras.metrics.Precision(name='precision')
recall = tf.keras.metrics.Recall(name='recall')


optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5, epsilon=1e-08, clipnorm=1.0)


bert_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy',precision,recall])


In [14]:
history=bert_model.fit([train_inp,train_mask],train_label,batch_size=32,epochs=5,validation_data=([val_inp,val_mask],val_label),callbacks=[])


Epoch 1/5
452/452 [==============================] - 7724s 17s/step - loss: 0.4665 - accuracy: 0.8981 - precision: 0.9018 - recall: 0.8954 - val_loss: 0.3440 - val_accuracy: 0.9335 - val_precision: 0.9082 - val_recall: 0.9593
Epoch 2/5
452/452 [==============================] - 7776s 17s/step - loss: 0.3125 - accuracy: 0.9522 - precision: 0.9504 - recall: 0.9551 - val_loss: 0.6011 - val_accuracy: 0.9302 - val_precision: 0.9224 - val_recall: 0.9340
Epoch 3/5
452/452 [==============================] - 20158s 45s/step - loss: 0.2580 - accuracy: 0.9699 - precision: 0.9665 - recall: 0.9741 - val_loss: 0.7190 - val_accuracy: 0.9310 - val_precision: 0.9187 - val_recall: 0.9403
Epoch 4/5
452/452 [==============================] - 7596s 17s/step - loss: 0.2286 - accuracy: 0.9778 - precision: 0.9763 - recall: 0.9797 - val_loss: 0.8991 - val_accuracy: 0.9280 - val_precision: 0.9032 - val_recall: 0.9530
Epoch 5/5
452/452 [==============================] - 7599s 17s/step - loss: 0.2253 - accuracy: 

In [15]:
history.model

In [16]:
# history.history

In [17]:
bert_model.save(r'C:\Users\admin\Desktop\Major project\dataset\modelBert',save_format='tf')

INFO:tensorflow:Assets written to: C:\Users\admin\Desktop\Major project\dataset\modelBert\assets


INFO:tensorflow:Assets written to: C:\Users\admin\Desktop\Major project\dataset\modelBert\assets


In [18]:
bert_pickle_data={'data':history.history}

In [19]:
import pickle

# Save the trained model object to a file
with open(r'C:\Users\admin\Desktop\Major project\dataset\modelBert.pkl', 'wb') as f:
    pickle.dump(bert_pickle_data, f)
